In [1]:
import torch
import sys
import os
import os.path as osp
import pandas as pd
import numpy as np
from scipy import stats
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import ticker
import matplotlib as mpl
from   matplotlib.colors import LogNorm
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, \
balanced_accuracy_score, accuracy_score, auc
from sklearn import metrics
import scipy

In [2]:
root_dir = "/home/users/richras/Ge2Net_Repo"
os.chdir(root_dir)

In [3]:
os.environ['USER_PATH']='/home/users/richras/Ge2Net_Repo'
os.environ['USER_SCRATCH_PATH']="/scratch/users/richras"
os.environ['IN_PATH']='/scratch/groups/cdbustam/richras/data_in'
os.environ['OUT_PATH']='/scratch/groups/cdbustam/richras/data_out'
os.environ['LOG_PATH']='/scratch/groups/cdbustam/richras/logs/'

In [4]:
%load_ext autoreload
%autoreload 2
from src.utils.dataUtil import load_path, save_file, vcf2npy, getWinInfo, getValueBySelection
from src.utils.modelUtil import Params, load_model, convert_nVector
from src.utils.labelUtil import getSuperpopBins, repeat_pop_arr
from src.utils.decorators import timer
from src.main.evaluation import GcdLoss
import test

In [5]:
# Specify the dataset to be evaluated
labels_path = osp.join(os.environ['OUT_PATH'],'humans/labels/data_id_2_geo')
data_path = osp.join(os.environ['OUT_PATH'],'humans/labels/data_id_2_geo')
dataset_type='valid'

In [6]:
#load test data snps and labels
gens_to_ret=[0]
for i, gen in enumerate(gens_to_ret):
    print(f"Loading gen {gen}")
    curr_snps = load_path(osp.join(data_path, str(dataset_type) ,'gen_' + str(gen), 'mat_vcf_2d.npy'))
    print(f' snps data: {curr_snps.shape}')
    curr_vcf_idx = load_path(osp.join(data_path , str(dataset_type) ,'gen_' + str(gen) ,'mat_map.npy'))
    print(f' y_labels data :{curr_vcf_idx.shape}')

    if i>0:
        snps = np.concatenate((snps, curr_snps),axis=0)
        vcf_idx = np.concatenate((vcf_idx, curr_vcf_idx),axis=0)
    else:
        snps = curr_snps
        vcf_idx = curr_vcf_idx

Loading gen 0
 snps data: (1250, 317410)
 y_labels data :(1250, 317410)


In [7]:
# load train/ref data snps and labels
gen=0
dataset_type="train"
trainSnps_npy = load_path(osp.join(data_path, str(dataset_type) ,'gen_' + str(gen), 'mat_vcf_2d.npy'))
train_y_vcf = load_path(osp.join(data_path , str(dataset_type) ,'gen_' + str(gen) ,'mat_map.npy'))

In [8]:
# transform the data into right shape 
chmLen=snps.shape[1]
winSize=1000
newChmLen, nWin=getWinInfo(chmLen, winSize)
query_snps = snps[:,:newChmLen]
trainSnps_npy = trainSnps_npy[:,:newChmLen]

In [9]:
def transformData(trainSnps_npy, query_snps, train_y_vcf, vcf_idx, labels_path, evalGlobalAnc):
    trainSnps=trainSnps_npy
    inputSnps=query_snps
    query_labels = vcf_idx[:,0]
    if not evalGlobalAnc:
        trainSnps=trainSnps.reshape(-1,nWin, winSize)
        inputSnps=inputSnps.reshape(-1,nWin, winSize)
        query_labels = vcf_idx[:,:newChmLen].reshape(-1,nWin, winSize)
        query_labels = stats.mode(query_labels, axis=2)[0].squeeze(2)
    # get granular pop labels for train/ref and test dataset
    pop_sample_map=pd.read_csv(osp.join(labels_path, 'pop_sample_map.tsv'),sep="\t")
    pop_arr=repeat_pop_arr(pop_sample_map)
    granular_pop_dict = load_path(osp.join(labels_path, 'granular_pop.pkl'), en_pickle=True)
    superpop_dict=load_path(osp.join(labels_path, 'superpop.pkl'), en_pickle=True)
    rev_pop_dict={v:k for k,v in granular_pop_dict.items()}
    idx2gp_dict={k:v for k,v in zip(pop_arr[:,1],pop_arr[:,2])}
    idx2sp_dict={k:v for k,v in zip(pop_arr[:,1],pop_arr[:,3])}
    trainGranularPops=np.vectorize(idx2gp_dict.get)(train_y_vcf[:,0])
    trainSuperPops=np.vectorize(idx2sp_dict.get)(train_y_vcf[:,0])
    if not evalGlobalAnc:
        trainGranularPops=trainGranularPops[:,np.newaxis, np.newaxis].astype("int16")
        trainSuperPops=trainSuperPops[:,np.newaxis, np.newaxis].astype("int16")
    uniqueGranularPops=np.unique(np.array(list(idx2gp_dict.values())))
    uniqueSuperPops=np.unique(np.array(list(idx2sp_dict.values())))
    query_gp = np.vectorize(idx2gp_dict.get)(query_labels)
    query_sp = np.vectorize(idx2sp_dict.get)(query_labels)
    idx2lbl_dict=load_path(osp.join(labels_path, 'labels.pkl'), en_pickle=True)
    gp2labels_lat_dict={idx2gp_dict[k]:idx2lbl_dict[k][0] for k in pop_arr[:,1]}
    gp2labels_long_dict={idx2gp_dict[k]:idx2lbl_dict[k][1] for k in pop_arr[:,1]}
    return uniqueGranularPops, uniqueSuperPops, trainGranularPops, trainSuperPops, query_gp, query_sp, \
            trainSnps, inputSnps, gp2labels_lat_dict, gp2labels_long_dict

In [10]:
@timer
def DistSimilarity(inputSnps, refTarget, distType="L2"):
    """
    Computes L2 distance of each window with each window of target
    and returns the sample from the refTarget that it is most closest to
    Input:
        inputSnps: tensor (num_query_samples x n_win x win_size)
        refTarget: tensor (num_ref_anc x n_win x win_size)
    Returns:
        preds: tensor (num_query_samples,) indices of target that it is closest to
    """
    
    matrix1 = np.repeat(inputSnps[:,np.newaxis,...], refTarget.shape[0], axis=1)
    matrix2 = refTarget[np.newaxis,...]
    if distType=="L2":
        distMatrix = np.sum(np.square(matrix1-matrix2), axis=-1) # num_query_samples x num_ref_anc x n_win 
    elif distType=="hamming":
        print(f"inputSnps shape, refTarget shape:{inputSnps.shape}, {refTarget.shape}")
        distFunc = np.vectorize(scipy.spatial.distance.cdist, signature='(m,n),(k,n)->(m,k)', excluded=['metric'])
        distMatrix = distFunc(inputSnps, refTarget, metric="hamming")
    print(f"distance Matrix shape:{distMatrix.shape}")
    idx=np.argmin(distMatrix, axis=1) #num_query_anc
    return idx

In [21]:
@timer
def getPredsbySimilarity(uniquePops, trainPoplabels, trainSnps, inputSnps, evalGlobalAnc, distType="L2"):
    PopMask=np.zeros((uniquePops.shape[0], trainPoplabels.shape[0],1), dtype='int8') if evalGlobalAnc \
    else np.zeros((uniquePops.shape[0], trainPoplabels.shape[0],1,1), dtype='int8')
    if evalGlobalAnc:trainPoplabels=trainPoplabels[...,np.newaxis]
    for i in uniquePops:
        PopMask[i,...]=trainPoplabels==i
    trainSnps=trainSnps[np.newaxis,...]
    print(trainSnps.shape, PopMask.shape)
    refTarget_tmp=np.sum(trainSnps*PopMask, axis=1)
    if distType=="hamming":
        refTarget_tmp=np.round(refTarget_tmp)
    print(refTarget_tmp.shape)
    refTarget=refTarget_tmp/np.sum(PopMask, axis=1) 
    print(f"refTarget shape:{refTarget.shape}")
    return DistSimilarity(inputSnps, refTarget, distType)

In [12]:
#calculate gcd between coordinates mapped to testPred and true coordinates of testPred

def gp2NVector(PredPops, gp2labels_lat_dict, gp2labels_long_dict):
    Lat = np.vectorize(gp2labels_lat_dict.get)(PredPops)
    Long = np.vectorize(gp2labels_long_dict.get)(PredPops)
    nVector = convert_nVector(Lat, Long)
    return nVector

def getAcc(PredPops, TargetPops):
    if len(TargetPops.shape)==2:
        acc = len(PredPops[PredPops==TargetPops])*100/(TargetPops.shape[0]*TargetPops.shape[1])
    elif len(TargetPops.shape)==1:
        acc = len(PredPops[PredPops==TargetPops])*100/(TargetPops.shape[0])
    return acc

In [13]:
# compute for granular pops 
def getMetrics(uniquePops, trainPops, trainSnps, inputSnps, query_pop, evalGlobalAnc, labelType="granular_pop"):
    metrics={}
    PredPops=getPredsbySimilarity(uniquePops, trainPops, trainSnps, inputSnps, evalGlobalAnc)
    if labelType=="granular_pop":
        PredNVector = gp2NVector(PredPops, gp2labels_lat_dict, gp2labels_long_dict)
        testNVector = gp2NVector(query_gp, gp2labels_lat_dict, gp2labels_long_dict)
        print(f"PredNVector.shape, testNVector.shape:{PredNVector.shape}, {testNVector.shape}")
        gcdObj=GcdLoss()
        metrics['gcd']=np.mean(gcdObj.rawGcd(PredNVector, testNVector))
        print(f"gcd:{metrics['gcd']}Km")
    metrics['acc'] = getAcc(PredPops, query_pop)
    print(f"Classification accuracy:{metrics['acc']}")
    return metrics

In [22]:
# get global ancestry
for evalGlobalAnc in [True, False]:
    uniqueGranularPops, uniqueSuperPops, trainGranularPops, trainSuperPops, query_gp, query_sp, \
    trainSnps, inputSnps, gp2labels_lat_dict, gp2labels_long_dict = \
    transformData(trainSnps_npy, query_snps, train_y_vcf, vcf_idx, labels_path, evalGlobalAnc)
    print(f" query_gp shape :{query_gp.shape}")
    print(f"trainGranularPops.shape, trainSuperPops.shape:{trainGranularPops.shape}, {trainSuperPops.shape}")
    if evalGlobalAnc:
        print(f"*******global granular anc metrics*******")
        globalGranularMetrics=getMetrics(uniqueGranularPops, trainGranularPops, trainSnps, inputSnps, query_gp, evalGlobalAnc=True,\
                                         labelType="granular_pop")
        print(f"*******global super anc metrics*******")
        globalSuperMetrics=getMetrics(uniqueSuperPops, trainSuperPops, trainSnps, inputSnps, query_sp, evalGlobalAnc=True,\
                                         labelType="super_pop")
    else:
        print(f"*******local granular anc metrics*******")
        GranularMetrics=getMetrics(uniqueGranularPops, trainGranularPops, trainSnps, inputSnps, query_gp, evalGlobalAnc=False,\
                                         labelType="granular_pop")
        print(f"*******local super anc metrics*******")
        SuperMetrics=getMetrics(uniqueSuperPops, trainSuperPops, trainSnps, inputSnps, query_sp, evalGlobalAnc=False,\
                                         labelType="super_pop")

 query_gp shape :(1250,)
trainGranularPops.shape, trainSuperPops.shape:(4116,), (4116,)
*******global granular anc metrics*******
(1, 4116, 317000) (136, 4116, 1)
(136, 317000)
refTarget shape:(136, 317000)
distance Matrix shape:(1250, 136)
Finished 'DistSimilarity' in 702.6993 secs
Finished 'getPredsbySimilarity' in 1098.8307 secs
PredNVector.shape, testNVector.shape:(1250, 3), (1250, 3)
gcd:871.9780957715071Km
Classification accuracy:47.6
*******global super anc metrics*******
(1, 4116, 317000) (7, 4116, 1)
(7, 317000)
refTarget shape:(7, 317000)
distance Matrix shape:(1250, 7)
Finished 'DistSimilarity' in 26.8546 secs
Finished 'getPredsbySimilarity' in 46.8522 secs
Classification accuracy:95.04
 query_gp shape :(1250, 317)
trainGranularPops.shape, trainSuperPops.shape:(4116, 1, 1), (4116, 1, 1)
*******local granular anc metrics*******
(1, 4116, 317, 1000) (136, 4116, 1, 1)
(136, 317, 1000)
refTarget shape:(136, 317, 1000)
distance Matrix shape:(1250, 136, 317)
Finished 'DistSimilari

In [23]:
globalGranularMetrics, globalSuperMetrics, GranularMetrics, SuperMetrics

({'acc': 47.6, 'gcd': 871.9780957715071},
 {'acc': 95.04},
 {'acc': 2.606687697160883, 'gcd': 5201.505430378055},
 {'acc': 36.926940063091486})

In [31]:
df_summary=pd.DataFrame(columns=['ClassificationAccuracy', 'Gcd', 'LabelType', 'PopType', 'DistType', 'gen', 'chm', 'DatasetType'])

df_summary.loc[0]=[globalGranularMetrics['acc'], globalGranularMetrics['gcd'], 'Granular_Pop', 'Global', 'L2', gen, '22', 'Single_Ancestry==1']
df_summary.loc[1]=[globalSuperMetrics['acc'], "N/A", 'Super_Pop', 'Global', 'L2', gen, '22', 'Single_Ancestry==1']
df_summary.loc[2]=[GranularMetrics['acc'], GranularMetrics['gcd'], 'Granular_Pop', 'Local', 'L2', gen, '22', 'Single_Ancestry==1']
df_summary.loc[3]=[SuperMetrics['acc'], "N/A", 'Super_Pop', 'Local', 'L2', gen, '22', 'Single_Ancestry==1']

In [32]:
df_summary

,ClassificationAccuracy,Gcd,LabelType,PopType,DistType,gen,chm,DatasetType
0,47.600000,871.978,Granular_Pop,Global,L2,0,22,Single_Ancestry==1
1,95.040000,N/A,Super_Pop,Global,L2,0,22,Single_Ancestry==1
2,2.606688,5201.51,Granular_Pop,Local,L2,0,22,Single_Ancestry==1
3,36.926940,N/A,Super_Pop,Local,L2,0,22,Single_Ancestry==1
